In [1]:
import pandas as pd
import numpy as np

The dataset is from https://www.kaggle.com/sonalisheth/in-depth-analysis-on-sales-dataset/notebook

In [2]:
!kaggle datasets download -d knightbearr/sales-product-data


  0%|          | 0.00/2.39M [00:00<?, ?B/s]
 42%|████▏     | 1.00M/2.39M [00:01<00:01, 807kB/s]
 84%|████████▎ | 2.00M/2.39M [00:02<00:00, 1.10MB/s]
100%|██████████| 2.39M/2.39M [00:02<00:00, 1.24MB/s]
100%|██████████| 2.39M/2.39M [00:02<00:00, 1.13MB/s]


In [3]:
# importing required modules
from zipfile import ZipFile

# specifying the zip file name
file_name = "sales-product-data.zip"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
	# printing all the contents of the zip file
	zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

File Name                                             Modified             Size
Sales_April_2019.csv                           2021-11-04 19:38:14      1595953
Sales_August_2019.csv                          2021-11-04 19:38:14      1043593
Sales_December_2019.csv                        2021-11-04 19:38:14      2181642
Sales_February_2019.csv                        2021-11-04 19:38:14      1046495
Sales_January_2019.csv                         2021-11-04 19:38:14       843098
Sales_July_2019.csv                            2021-11-04 19:38:14      1248753
Sales_June_2019.csv                            2021-11-04 19:38:14      1182508
Sales_March_2019.csv                           2021-11-04 19:38:14      1323497
Sales_May_2019.csv                             2021-11-04 19:38:14      1443965
Sales_November_2019.csv                        2021-11-04 19:38:14      1534677
Sales_October_2019.csv                         2021-11-04 19:38:14      1770338
Sales_September_2019.csv                

In [4]:
#merge all the dataset into a single csv
import glob
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False)

Let's see how the data looks!

In [5]:
combined_csv #show data

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
11681,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
11682,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
11683,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
11684,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [6]:
# getting the information
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186850 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 10.0+ MB


we can see there are some missing values and wrong data types. we will fix it soon

In [7]:
# unique values of these variables
categorical = combined_csv.select_dtypes(['category', 'object']).columns # getting the Uniqueness catrgorical variable
for col in categorical:
    print('{} : {} unique value(s)'.format(col, combined_csv[col].nunique()))

Order ID : 178438 unique value(s)
Product : 20 unique value(s)
Quantity Ordered : 10 unique value(s)
Price Each : 24 unique value(s)
Order Date : 142396 unique value(s)
Purchase Address : 140788 unique value(s)


In [8]:
# sum of missing data points
combined_csv.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [9]:
combined_csv.isnull().head(3)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,False,False,False,False,False,False
1,True,True,True,True,True,True
2,False,False,False,False,False,False


we can see all of the missing value, they are at the same row, so we can't make imputation out of it. Let's just drop the missing value and I think this will not affect the data since the data is big enough.

In [10]:
# let's drop the rows of NaN data!
combined_csv = combined_csv.dropna(how='all')
# okay, let's check it again!
"NaN Value:"
combined_csv[combined_csv.isna().any(axis=1)]
# future warning! ValueError: invalid literal for int() with base 10: 'Or'
"Clean Future Warnings:"
combined_csv = combined_csv[combined_csv['Order Date'].str[0:2] != 'Or']
combined_csv.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"


In [11]:
# # drop the rows of NaN data!
# combined_csv.dropna(inplace=True)

Let's change data types of Quantity Ordered column, Price Each column and Order Date column

In [12]:
# convert the data to numerical
combined_csv['Quantity Ordered'], combined_csv['Price Each'] = combined_csv['Quantity Ordered'].astype('int64'), combined_csv['Price Each'].astype('float')

# convert using to_datetime() 
combined_csv['Order Date'] = pd.to_datetime(combined_csv['Order Date'])

combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185950 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  object        
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int64         
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 9.9+ MB


In [13]:
combined_csv.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"


This is some point that we have.
We have total 186850 records and 6 columns cateogircal type The total of missing value that we have is 0.29167%

- Order ID : 178438 unique value(s)
- Product : 20 unique value(s)
- Quantity Ordered : 10 unique value(s)
- Price Each : 24 unique value(s)
- Order Date : 142396 unique value(s)
- Purchase Address : 140788 unique value(s)

### Data Preparation

Add Month, Hour, Minute, Sales, Cities Column

In [14]:
def augment_data(data):
    
    """
    Adding new features to
    our data, adding Month Data,
    Hour Data, Minute Data, Sales Data,
    and Cities Column
    
    Returning:
        data with new features
    """
    
    # funtction to get the city in the data
    def get_city(address):
        return address.split(',')[1]
    
    # funtction to get the state in the data
    def get_state(address):
        return address.split(',')[2].split(' ')[1]

    # let's get the year data in order date column
    data['Year'] = data['Order Date'].dt.year
    
    # let's get the month data in order date column
    data['Month'] = data['Order Date'].dt.month
    
    # let's get the houe data in order date column
    data['Hour'] = data['Order Date'].dt.hour 
    
    # let's get the minute data in order date column
    data['Minute'] = data['Order Date'].dt.minute 
    
    # let's make the sales column by multiplying the quantity ordered colum with price each column
    data['Sales'] = data['Quantity Ordered'] * data['Price Each'] 
    
    # let's get the cities data in order date column
    data['Cities'] = data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})") 
    
    return data # returning data

# and see it
sales_data = augment_data(combined_csv)
sales_data

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Year,Month,Hour,Minute,Sales,Cities
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",2019,4,8,46,23.90,Dallas (TX)
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",2019,4,22,30,99.99,Boston (MA)
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",2019,4,14,38,600.00,Los Angeles (CA)
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",2019,4,14,38,11.99,Los Angeles (CA)
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",2019,4,9,27,11.99,Los Angeles (CA)
...,...,...,...,...,...,...,...,...,...,...,...,...
11681,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",2019,9,20,56,8.97,Los Angeles (CA)
11682,259354,iPhone,1,700.00,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",2019,9,16,0,700.00,San Francisco (CA)
11683,259355,iPhone,1,700.00,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",2019,9,7,39,700.00,San Francisco (CA)
11684,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",2019,9,17,30,379.99,San Francisco (CA)


In [15]:
#upload the cleaned data
sales_data.to_csv('D:\Course\Metabase\Sales Product/sales_data.csv', index=False)